# Data Extraction
A partir de um arquivo .txt, extrai a tabela contendo as datas e os números de cada sorteio.

time: 19.7 s (started: 2023-07-23 20:57:35 +00:00)


# Data Transformation
A partir da tabela que contém os sorteios, cria uma segunda contendo dados estatísticos e outras informações adicionais sobre os sorteios. Esta etapa será o equivalente à engenharia de atributos, mas ao contrário do que é feito normalmente, será feita antes da análise exploratória porque o conjunto de dados precisa ser criado primeiro.

In [57]:
%pip install ipython-autotime  --upgrade

time: 5.03 s (started: 2023-07-23 20:57:55 +00:00)


In [58]:
from google.colab import drive, files
import joblib
import pandas as pd
drive.mount('/content/drive', force_remount=True)
%load_ext autotime

Mounted at /content/drive
The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 3.95 s (started: 2023-07-23 20:58:00 +00:00)


In [59]:
raffles = joblib.load("/content/drive/MyDrive/datasets/loterias/loterias-lotofacil.pkl")
#dataset['Data_Sorteio'] = pd.to_datetime(dataset['Data_Sorteio'], format="%d/%m/%Y")
#dataset = dataset.set_index('Data_Sorteio')
raffles = raffles.drop(['Data_Sorteio'], axis=1)
raffles.head()

,Bola1,Bola2,Bola3,Bola4,Bola5,Bola6,Bola7,Bola8,Bola9,Bola10,Bola11,Bola12,Bola13,Bola14,Bola15
0,2.0,3.0,5.0,6.0,9.0,10.0,11.0,13.0,14.0,16.0,18.0,20.0,23.0,24.0,25.0
1,1.0,4.0,5.0,6.0,7.0,9.0,11.0,12.0,13.0,15.0,16.0,19.0,20.0,23.0,24.0
2,1.0,4.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,14.0,16.0,17.0,20.0,23.0,24.0
3,1.0,2.0,4.0,5.0,8.0,10.0,12.0,13.0,16.0,17.0,18.0,19.0,23.0,24.0,25.0
4,1.0,2.0,4.0,8.0,9.0,11.0,12.0,13.0,15.0,16.0,19.0,20.0,23.0,24.0,25.0


time: 139 ms (started: 2023-07-23 20:58:04 +00:00)


In [60]:
from sklearn.base     import BaseEstimator, TransformerMixin
from sklearn.compose  import ColumnTransformer
from sklearn.pipeline import Pipeline

import itertools
import numpy as np

time: 865 µs (started: 2023-07-23 20:58:04 +00:00)


In [61]:
class DescriptiveStatistics(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self

  def fit_transform(self, X, y=None):
    self.fit(X, y)
    return self.transform(X, y)

  def transform(self, X, y=None):
    X_ = X.copy()
    X_['mean'] = X_.apply(lambda y: y.mean(), axis=1)
    X_['std']  = X_.apply(lambda y: y.std(), axis=1)
    X_['min']  = X_.apply(lambda y: y.min(), axis=1)
    X_['max']  = X_.apply(lambda y: y.max(), axis=1)
    return X_

time: 4.42 ms (started: 2023-07-23 20:58:04 +00:00)


In [62]:
class EvenAmount(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self

  def fit_transform(self, X, y=None):
    self.fit(X, y)
    return self.transform(X, y)

  def get_feature_names_out(self):
    pass

  def even_amount(self, raffle):
    raffle_size = 15
    even = itertools.filterfalse(lambda x: x % 2 != 0, raffle.iloc[:raffle_size])
    return len(list(even))

  def transform(self, X, y=None):
    X = X.copy()
    raffle_size = 15
    X['even_amount'] = X.apply(self.even_amount, axis=1)
    return X

time: 1.73 ms (started: 2023-07-23 20:58:05 +00:00)


In [63]:
class Quartiles(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self

  def fit_transform(self, X, y=None):
    self.fit(X, y)
    return self.transform(X, y)

  def get_feature_names_out(self):
    pass

  def transform(self, X, y=None):
    X_ = X.iloc[:, 0: 15].copy()
    X['quartile_1_5']   = X_.apply(lambda y: ((y >= 1) & (y <= 5)).sum(), axis=1)
    X['quartile_6_10']  = X_.apply(lambda y: ((y >= 6) & (y <= 10)).sum(), axis=1)
    X['quartile_11_15'] = X_.apply(lambda y: ((y >= 11) & (y <= 15)).sum(), axis=1)
    X['quartile_16_20'] = X_.apply(lambda y: ((y >= 16) & (y <= 20)).sum(), axis=1)
    X['quartile_21_25'] = X_.apply(lambda y: ((y >= 21) & (y <=25)).sum(), axis=1)
    return X

time: 2.16 ms (started: 2023-07-23 20:58:05 +00:00)


In [64]:
class RaffleRemover(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self

  def fit_transform(self, X, y=None):
    self.fit(X, y)
    return self.transform(X, y)

  def transform(self, X, y=None):
    return X.iloc[:, 15:]

time: 1.07 ms (started: 2023-07-23 20:58:05 +00:00)


In [65]:
class RepeatedOne(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self

  def fit_transform(self, X, y=None):
    self.fit(X, y)
    return self.transform(X, y)

  def match_numbers(self, X):
      raffle_x = set(X[['Bola1', 'Bola2', 'Bola3', 'Bola4', 'Bola5', 'Bola6', 'Bola7', 'Bola8', 'Bola9', 'Bola10', 'Bola11', 'Bola12', 'Bola13', 'Bola14', 'Bola15']])
      raffle_y = set(X[['lag1_Bola1', 'lag1_Bola2', 'lag1_Bola3', 'lag1_Bola4', 'lag1_Bola5', 'lag1_Bola6', 'lag1_Bola7', 'lag1_Bola8', 'lag1_Bola9', 'lag1_Bola10', 'lag1_Bola11', 'lag1_Bola12', 'lag1_Bola13', 'lag1_Bola14', 'lag1_Bola15']])
      return len(raffle_x.intersection(raffle_y))

  def transform(self, X, y=None):
    X_ = X.copy()
    for name in X_.columns:
      X_['lag1_' + name] = X_[name].shift(1)
    X['repeated'] = X_.apply(self.match_numbers, axis=1)
    return X

time: 1.81 ms (started: 2023-07-23 20:58:05 +00:00)


In [111]:
class QuartilesPatternMapping(BaseEstimator, TransformerMixin):
  """
  Codifica cada sequência de valores presentes nos atributos quartile_1_5 até quartile_21_25 em apenas um número.
  Assim, é possível criar um atributo substituindo os cinco.
  """
  def fit(self, X, y=None):
    X_ = X[['quartile_1_5', 'quartile_6_10', 'quartile_11_15', 'quartile_16_20', 'quartile_21_25']]
    self.quartiles = list()
    X_.apply(lambda y: self.quartiles.append(np.array2string(y.to_numpy())), axis=1) # transforma cada sequência de cinco números em uma string
    patterns_unique = {item for item in self.quartiles} # set comprehension
    self.patterns_map = {value: index for index, value in enumerate(patterns_unique)} # dicionário padrão:id
    return self

  def fit_transform(self, X, y=None):
    self.fit(X, y)
    return self.transform(X, y)

  def transform(self, X, y=None):
    X['quartiles'] = pd.Series(self.quartiles).replace(self.patterns_map)
    X = X.drop(['quartile_1_5', 'quartile_6_10', 'quartile_11_15', 'quartile_16_20', 'quartile_21_25'], axis=1)
    return X

time: 1.62 ms (started: 2023-07-23 21:34:19 +00:00)


## Pipeline completo

In [ ]:
steps = [('descriptivestatistics', DescriptiveStatistics()),
         ('repeated', RepeatedOne()),
         ('evenodd', EvenAmount()),
         ('quartiles', Quartiles()),
         ('raffleremover', RaffleRemover()),
         ('quartilespatternmapping', QuartilesPatternMapping())]

pipeline_full = Pipeline(steps=steps)
dataset = pipeline_full.fit_transform(raffles)
dataset.head()